In [7]:
import pandas as pd
import time
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from concurrent.futures import ThreadPoolExecutor, as_completed

# ChromeDriver 설정
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")  # 브라우저를 최대화
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

wait = WebDriverWait(driver, 10)

# 사용자 티어 매핑
tier_map = {
    0: "Unrated",
    1: "Bronze V", 2: "Bronze IV", 3: "Bronze III", 4: "Bronze II", 5: "Bronze I",
    6: "Silver V", 7: "Silver IV", 8: "Silver III", 9: "Silver II", 10: "Silver I",
    11: "Gold V", 12: "Gold IV", 13: "Gold III", 14: "Gold II", 15: "Gold I",
    16: "Platinum V", 17: "Platinum IV", 18: "Platinum III", 19: "Platinum II", 20: "Platinum I",
    21: "Diamond V", 22: "Diamond IV", 23: "Diamond III", 24: "Diamond II", 25: "Diamond I",
    26: "Ruby V", 27: "Ruby IV", 28: "Ruby III", 29: "Ruby II", 30: "Ruby I",
    31: "Master"
}

# 태그 한글 번역 함수 (displayNames 배열에서 "ko" 언어를 찾고 없으면 영어 사용)
def translate_tags(tags):
    translated_tags = []
    for tag in tags:
        ko_name = None
        for name in tag.get('displayNames', []):
            if name.get('language') == 'ko':
                ko_name = name.get('name')
                break
        translated_tags.append(ko_name if ko_name else tag.get('key', 'Unknown'))
    return translated_tags

def get_user_info_via_api(handle, retries=2, delay=2):
    url = f"https://solved.ac/api/v3/user/show?handle={handle}"
    headers = {
        "Accept": "application/json",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)" \
                      " Chrome/90.0.4430.93 Safari/537.36"
        # "Authorization": "Bearer YOUR_API_KEY"  # 필요 시 추가
    }

    for attempt in range(retries):
        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                return response.json()
            elif response.status_code == 429:
                print(f"API 호출 제한 발생 (429). 응답 내용: {response.text}")
                print(f"대기 후 재시도 중... ({attempt + 1}/{retries})")
                time.sleep(delay)  # 지연 시간
            else:
                print(f"사용자 정보 API 호출 오류: {response.status_code}, 응답 내용: {response.text}")
                return None
        except requests.exceptions.RequestException as e:
            print(f"API 요청 실패: {e}. 재시도 중... ({attempt + 1}/{retries})")
            time.sleep(delay)
    return None

def get_top_100_problems_via_api(handle, retries=2, delay=2):
    url = f"https://solved.ac/api/v3/user/top_100?handle={handle}"
    headers = {
        "Accept": "application/json",
        "x-solvedac-language": "ko"  # 응답 언어를 한국어로 설정
    }

    for attempt in range(retries):
        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                problem_stats = response.json()  # 응답은 metadata와 items를 포함한 dict
                if 'items' in problem_stats:
                    return problem_stats['items']  # items 리스트 반환
                else:
                    print(f"예상치 못한 응답 형식: {problem_stats}")
                    return []
            elif response.status_code == 429:  # Too Many Requests
                print(f"API 호출 제한 발생 (429). 대기 후 재시도 중... ({attempt + 1}/{retries})")
                time.sleep(delay)
            else:
                print(f"문제 API 호출 오류: {response.status_code}, 응답 내용: {response.text}")
                return None
        except requests.exceptions.RequestException as e:
            print(f"API 요청 실패: {e}. 재시도 중... ({attempt + 1}/{retries})")
            time.sleep(delay)  # 재시도 간 대기 시간을 늘림
    return None  # 재시도 실패 시 None 반환

# 스크롤을 끝까지 내리는 함수 (동적으로 사용자 이름 가져오기 위해 사용)
def scroll_to_end(driver):
    SCROLL_PAUSE_TIME = 1
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

def get_user_names(driver):
    users = driver.find_elements(By.CSS_SELECTOR, "b[class='css-oo6qmd']")  # 사용자 이름

    user_list = []
    for user in users:
        user_name = user.text
        user_profile_url = user.find_element(By.XPATH, "..").get_attribute('href')  # 프로필 URL
        
        user_list.append({
            "name": user_name,
            "profile_url": user_profile_url,
            "handle": user_profile_url.split("/")[-1]  # 사용자 handle 추출
        })
    
    return user_list

def add_top_100_to_dataframe(user, problems):
    tier_difference_count = 0 
    for problem in problems:
        # 필요한 키가 있는지 확인
        problem_level = problem.get('level', 0)
        solved_count = problem.get('acceptedUserCount', 0)  # 'solved' 대신 'acceptedUserCount' 사용
        
        # 문제를 푼 사람이 없으면 건너뜀
        if solved_count == 0:
            continue
        
        # 문제 링크 생성
        problem_link = f"https://www.acmicpc.net/problem/{problem.get('problemId', 'Unknown')}"
        
        # 티어 차이 계산
        user_tier = user.get('tier', 0)
        tier_difference = problem_level - user_tier
        
        if tier_difference > 4:
            tier_difference_count += 1

        if tier_difference_count >= 4:
            print(f"사용자 {user['name']}는 상위 티어 문제를 여러 개 풀었음. 부계정으로 의심됨, 수집 중단.")
            return False
        
        # 태그 번역
        tags = translate_tags(problem.get('tags', []))
        translated_tags = ', '.join(tags) if tags else 'N/A'
        
        # 데이터 추가
        all_data.append({
            "티어": tier_map.get(user['tier'], "Unknown"),  # 사용자 티어 텍스트로 변환
            "닉네임": user['name'],
            "푼 문제 수": user['solvedCount'],  # API에서 가져온 푼 문제 수 사용
            "문제 번호": problem.get('problemId', "Unknown"),  # Default to "Unknown" if missing
            "문제 티어": tier_map.get(problem_level, "Unknown"),  # 문제 레벨을 텍스트로 변환
            "문제 이름": problem.get('titleKo', "Unknown"),  # Default to "Unknown" if missing
            "문제 푼 사람 수": solved_count,
            "문제 태그": translated_tags,  # 태그 정보 추가
            "문제 링크": problem_link  # 문제 링크 추가
        })
    
    return True

# 최종 데이터를 담을 리스트
all_data = []

# 주기적으로 데이터를 백업하는 함수
def save_backup(df):
    df.to_csv("solved_ac_data_backup.csv", index=False)
    print("중간 데이터를 solved_ac_data_backup.csv에 저장했습니다.")

# 최종 데이터를 저장하는 함수
def save_final(df):
    df.to_csv("solved_ac_data(Platinum I ~ Platinum III).csv", index=False)
    print("최종 데이터를 solved_ac_data.csv에 저장했습니다.")

def fetch_user_data(user):
    try:
        # 사용자 정보 가져오기
        user_info = get_user_info_via_api(user['handle'])
        if user_info:
            user['tier'] = user_info['tier']
            user['solvedCount'] = user_info['solvedCount']

            # 푼 문제 수가 30 이상인 사용자만 수집
            if user['solvedCount'] < 30:
                print(f"사용자 {user['name']}의 문제 수가 30 미만, 수집하지 않음.")
                return None

            # 상위 100 문제 가져오기
            top_100_problems = get_top_100_problems_via_api(user['handle'])
            if top_100_problems:
                # 티어 차이가 4 이상인 문제가 2개 이상이면 부계정으로 간주
                if not add_top_100_to_dataframe(user, top_100_problems):
                    return None

        # API 요청 후 짧은 시간 대기 (1초 ~ 2초)
        time.sleep(1)
        
        return user
    except Exception as e:
        print(f"에러 발생: {e}")
        return None

# 문제 수집 후 데이터프레임에 추가하는 부분
try:
    for page in range(38, 26, -1):
        url = f"https://solved.ac/ranking/tier?page={page}"
        try:
            driver.get(url)

            print(f"페이지 {page} 로딩 중...")

            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".css-1fudcfm")))  # 페이지가 로드될 때까지 대기
            scroll_to_end(driver)  # 스크롤 끝까지 내리기

            # 사용자 목록 가져오기 (동적 크롤링: 이름만 가져옴)
            user_list = get_user_names(driver)

            # 병렬로 API 호출 처리 (각 사용자별로 API 호출)
            with ThreadPoolExecutor(max_workers=20) as executor:
                futures = [executor.submit(fetch_user_data, user) for user in user_list]
                for future in as_completed(futures):
                    result = future.result()
                    if result:
                        print(f"사용자 {result['name']}의 문제 수집 완료.")
                    # time.sleep(0.5)  # 각 요청 후 추가 대기 시간

            print(f"페이지 {page} 크롤링 완료.")

            # 1페이지마다 백업 저장
            if page % 1 == 0:
                df = pd.DataFrame(all_data)  # 리스트를 데이터프레임으로 변환
                save_backup(df)

            time.sleep(2)  # 페이지 간 대기 시간 설정 (서버 과부하 방지)

        except Exception as page_error:
            print(f"페이지 {page}에서 에러 발생: {page_error}")

    # 모든 크롤링이 정상적으로 완료된 경우, 최종 데이터 저장
    df = pd.DataFrame(all_data)
    save_final(df)

except Exception as e:
    print(f"에러 발생: {e}")
    # 에러가 발생할 경우 현재까지 수집한 데이터를 백업 저장
    df = pd.DataFrame(all_data)
    save_backup(df)

driver.quit()


페이지 38 로딩 중...
사용자 bjwj5505는 상위 티어 문제를 여러 개 풀었음. 부계정으로 의심됨, 수집 중단.
사용자 pinebananais는 상위 티어 문제를 여러 개 풀었음. 부계정으로 의심됨, 수집 중단.
사용자 dkvmflzk98의 문제 수집 완료.
사용자 phyourak0529의 문제 수집 완료.
사용자 qkrwndnjs는 상위 티어 문제를 여러 개 풀었음. 부계정으로 의심됨, 수집 중단.
사용자 k1671k의 문제 수집 완료.
사용자 uss425의 문제 수집 완료.
사용자 jh280722의 문제 수집 완료.
사용자 brant2762의 문제 수집 완료.
사용자 an3735297의 문제 수집 완료.
사용자 abcdefg의 문제 수집 완료.
사용자 kjhonggg의 문제 수집 완료.
사용자 tpdud3072의 문제 수집 완료.
사용자 jongwook123의 문제 수집 완료.
사용자 lsh9034의 문제 수집 완료.
사용자 jerry3972의 문제 수집 완료.
사용자 bysu의 문제 수집 완료.
사용자 jaegoomon의 문제 수집 완료.
사용자 tph00300의 문제 수집 완료.
사용자 ace913의 문제 수집 완료.
사용자 kimhy4480의 문제 수집 완료.
사용자 jseo의 문제 수집 완료.
사용자 cmk5187의 문제 수집 완료.
사용자 coding_dana의 문제 수집 완료.
사용자 tjtkdgus45의 문제 수집 완료.
사용자 whites0701의 문제 수집 완료.
사용자 jinik9903의 문제 수집 완료.
사용자 sewon1407의 문제 수집 완료.
사용자 dlbae9613의 문제 수집 완료.
사용자 loftol의 문제 수집 완료.
사용자 tyg03485는 상위 티어 문제를 여러 개 풀었음. 부계정으로 의심됨, 수집 중단.
사용자 julysky의 문제 수집 완료.
사용자 holenet의 문제 수집 완료.
사용자 dlgprbs9441의 문제 수집 완료.
사용자 yskang의 문제 수집 완료.
사용자 braiden9377의 문제 수집 

In [8]:
df

,티어,닉네임,푼 문제 수,문제 번호,문제 티어,문제 이름,문제 푼 사람 수,문제 태그,문제 링크
0,Platinum II,bjwj5505,130,17510,Ruby V,Bigger Sokoban 40k,92,해 구성하기,https://www.acmicpc.net/problem/17510
1,Platinum II,bjwj5505,130,4004,Diamond I,쿠나이,41,"자료 구조, 구현, 우선순위 큐, 세그먼트 트리, 스위핑",https://www.acmicpc.net/problem/4004
2,Platinum II,bjwj5505,130,15930,Diamond I,Baby Seokhwan,13,"자료 구조, 퍼시스턴트 세그먼트 트리, 세그먼트 트리",https://www.acmicpc.net/problem/15930
3,Platinum II,dkvmflzk98,668,1616,Diamond III,드럼통 메시지,107,"오일러 경로, 그래프 이론",https://www.acmicpc.net/problem/1616
4,Platinum II,dkvmflzk98,668,13545,Diamond IV,수열과 쿼리 0,628,"mo's, 오프라인 쿼리, 누적 합, 제곱근 분할법",https://www.acmicpc.net/problem/13545
...,...,...,...,...,...,...,...,...,...
56403,Platinum I,kritias,1476,17428,Platinum V,K번째 괄호 문자열,207,"조합론, 다이나믹 프로그래밍, 수학",https://www.acmicpc.net/problem/17428
56404,Platinum I,kritias,1476,19541,Platinum V,역학 조사,373,"그리디 알고리즘, 구현, 시뮬레이션",https://www.acmicpc.net/problem/19541
56405,Platinum I,kritias,1476,20367,Platinum V,3 Slot Matching,61,다이나믹 프로그래밍,https://www.acmicpc.net/problem/20367
56406,Platinum I,kritias,1476,1016,Gold I,제곱 ㄴㄴ 수,9711,"수학, 정수론, 소수 판정, 에라토스테네스의 체",https://www.acmicpc.net/problem/1016


In [9]:
# Define the tier order from lowest to highest
tier_order = [
    'Not ratable Unrated',
    'Bronze V',
    'Bronze IV',
    'Bronze III',
    'Bronze II',
    'Bronze I',
    'Silver V',
    'Silver IV',
    'Silver III',
    'Silver II',
    'Silver I',
    'Gold V',
    'Gold IV',
    'Gold III',
    'Gold II',
    'Gold I',
    'Platinum V',
    'Platinum IV',
    'Platinum III',
    'Platinum II',
    'Platinum I',
    'Diamond V',
    'Diamond IV',
    'Diamond III',
    'Diamond II',
    'Diamond I',
    'Ruby V',
    'Ruby IV',
    'Ruby III',
    'Ruby II',
    'Ruby I'
]

# Create a mapping from tier names to numerical values
tier_mapping = {tier: index for index, tier in enumerate(tier_order)}

# Map '티어' (user tier) and '문제 티어' (problem tier) to numerical values
df['user_tier_num'] = df['티어'].map(tier_mapping)
df['problem_tier_num'] = df['문제 티어'].map(tier_mapping)

# # Compute the difference between problem tier and user tier
# df['tier_diff'] = df['problem_tier_num'] - df['user_tier_num']

# # Remove rows where the problem tier is 4 or more levels higher than the user tier
# df = df[df['tier_diff'] < 4]

# Optionally, drop the helper columns if you no longer need them
df = df.drop(columns=['user_tier_num', 'problem_tier_num'])
df = df.reset_index(drop=True)


In [10]:
df

,티어,닉네임,푼 문제 수,문제 번호,문제 티어,문제 이름,문제 푼 사람 수,문제 태그,문제 링크
0,Platinum II,bjwj5505,130,17510,Ruby V,Bigger Sokoban 40k,92,해 구성하기,https://www.acmicpc.net/problem/17510
1,Platinum II,bjwj5505,130,4004,Diamond I,쿠나이,41,"자료 구조, 구현, 우선순위 큐, 세그먼트 트리, 스위핑",https://www.acmicpc.net/problem/4004
2,Platinum II,bjwj5505,130,15930,Diamond I,Baby Seokhwan,13,"자료 구조, 퍼시스턴트 세그먼트 트리, 세그먼트 트리",https://www.acmicpc.net/problem/15930
3,Platinum II,dkvmflzk98,668,1616,Diamond III,드럼통 메시지,107,"오일러 경로, 그래프 이론",https://www.acmicpc.net/problem/1616
4,Platinum II,dkvmflzk98,668,13545,Diamond IV,수열과 쿼리 0,628,"mo's, 오프라인 쿼리, 누적 합, 제곱근 분할법",https://www.acmicpc.net/problem/13545
...,...,...,...,...,...,...,...,...,...
56403,Platinum I,kritias,1476,17428,Platinum V,K번째 괄호 문자열,207,"조합론, 다이나믹 프로그래밍, 수학",https://www.acmicpc.net/problem/17428
56404,Platinum I,kritias,1476,19541,Platinum V,역학 조사,373,"그리디 알고리즘, 구현, 시뮬레이션",https://www.acmicpc.net/problem/19541
56405,Platinum I,kritias,1476,20367,Platinum V,3 Slot Matching,61,다이나믹 프로그래밍,https://www.acmicpc.net/problem/20367
56406,Platinum I,kritias,1476,1016,Gold I,제곱 ㄴㄴ 수,9711,"수학, 정수론, 소수 판정, 에라토스테네스의 체",https://www.acmicpc.net/problem/1016


In [11]:
df.to_csv("solved_ac_data(Platinum I ~ Platinum III).csv", index=False)


In [16]:
df = pd.read_csv("solved_ac_data(Gold II ~ Platinum I).csv")
df

C:\Users\selah\AppData\Local\Temp\ipykernel_18232\303426539.py:1: DtypeWarning: Columns (2,3,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("solved_ac_data(Gold II ~ Platinum I).csv")


,티어,닉네임,푼 문제 수,문제 번호,문제 티어,문제 이름,문제 푼 사람 수,문제 태그,문제 링크
0,Gold II,qmfeksdh,329,11003,Platinum V,최솟값 찾기,8687,"자료 구조, 덱, 우선순위 큐",https://www.acmicpc.net/problem/11003
1,Gold II,qmfeksdh,329,11438,Platinum V,LCA 2,7040,"자료 구조, 최소 공통 조상, 희소 배열, 트리",https://www.acmicpc.net/problem/11438
2,Gold II,qmfeksdh,329,2042,Gold I,구간 합 구하기,13722,"세그먼트 트리, 자료 구조",https://www.acmicpc.net/problem/2042
3,Gold II,qmfeksdh,329,2357,Gold I,최솟값과 최댓값,9768,"세그먼트 트리, 자료 구조",https://www.acmicpc.net/problem/2357
4,Gold II,qmfeksdh,329,11505,Gold I,구간 곱 구하기,7464,"세그먼트 트리, 자료 구조",https://www.acmicpc.net/problem/11505
...,...,...,...,...,...,...,...,...,...
934494,Platinum I,kritias,1476,17428,Platinum V,K번째 괄호 문자열,207,"조합론, 다이나믹 프로그래밍, 수학",https://www.acmicpc.net/problem/17428
934495,Platinum I,kritias,1476,19541,Platinum V,역학 조사,373,"그리디 알고리즘, 구현, 시뮬레이션",https://www.acmicpc.net/problem/19541
934496,Platinum I,kritias,1476,20367,Platinum V,3 Slot Matching,61,다이나믹 프로그래밍,https://www.acmicpc.net/problem/20367
934497,Platinum I,kritias,1476,1016,Gold I,제곱 ㄴㄴ 수,9711,"수학, 정수론, 소수 판정, 에라토스테네스의 체",https://www.acmicpc.net/problem/1016


In [17]:
df = df.drop_duplicates()
df

,티어,닉네임,푼 문제 수,문제 번호,문제 티어,문제 이름,문제 푼 사람 수,문제 태그,문제 링크
0,Gold II,qmfeksdh,329,11003,Platinum V,최솟값 찾기,8687,"자료 구조, 덱, 우선순위 큐",https://www.acmicpc.net/problem/11003
1,Gold II,qmfeksdh,329,11438,Platinum V,LCA 2,7040,"자료 구조, 최소 공통 조상, 희소 배열, 트리",https://www.acmicpc.net/problem/11438
2,Gold II,qmfeksdh,329,2042,Gold I,구간 합 구하기,13722,"세그먼트 트리, 자료 구조",https://www.acmicpc.net/problem/2042
3,Gold II,qmfeksdh,329,2357,Gold I,최솟값과 최댓값,9768,"세그먼트 트리, 자료 구조",https://www.acmicpc.net/problem/2357
4,Gold II,qmfeksdh,329,11505,Gold I,구간 곱 구하기,7464,"세그먼트 트리, 자료 구조",https://www.acmicpc.net/problem/11505
...,...,...,...,...,...,...,...,...,...
934494,Platinum I,kritias,1476,17428,Platinum V,K번째 괄호 문자열,207,"조합론, 다이나믹 프로그래밍, 수학",https://www.acmicpc.net/problem/17428
934495,Platinum I,kritias,1476,19541,Platinum V,역학 조사,373,"그리디 알고리즘, 구현, 시뮬레이션",https://www.acmicpc.net/problem/19541
934496,Platinum I,kritias,1476,20367,Platinum V,3 Slot Matching,61,다이나믹 프로그래밍,https://www.acmicpc.net/problem/20367
934497,Platinum I,kritias,1476,1016,Gold I,제곱 ㄴㄴ 수,9711,"수학, 정수론, 소수 판정, 에라토스테네스의 체",https://www.acmicpc.net/problem/1016


In [18]:
df.reset_index(drop=True)
df
df.to_csv("solved_ac_data(Gold II ~ Platinum I).csv", index=False)
